<a href="https://colab.research.google.com/github/dudeurv/SAM_MRI/blob/main/mri_classes_to_two.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install nilearn nibabel SimpleITK

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Part 1: Setup
import os
import zipfile
import SimpleITK as sitk
import numpy as np
import nibabel as nib
import gdown

# Part 2: Data Preparation
base_path = '/content/drive/MyDrive/endonasal_mri_new'  # Change to the path of your folder
output_folder = '/content/drive/MyDrive/endonasal_mri_new/processed_mri_images'  # Change to your desired output folder path

os.makedirs(output_folder, exist_ok=True)  # Create the output folder

# Part 3: Processing Functions
def process_mri_with_itk(mri_folder, output_folder):
    ica_path = os.path.join(mri_folder, f"{os.path.basename(mri_folder)}_ICAs.nii.gz")
    pituitary_tumour_path = os.path.join(mri_folder, f"{os.path.basename(mri_folder)}_PituitaryAndTumour.nii.gz")

    # Load images
    ica_img = sitk.ReadImage(ica_path)
    pituitary_tumour_img = sitk.ReadImage(pituitary_tumour_path)

    # Convert to arrays for processing
    ica_array = sitk.GetArrayFromImage(ica_img)
    pituitary_tumour_array = sitk.GetArrayFromImage(pituitary_tumour_img)

    # Reassign the class of ICA from 1 to 2 and combine ICAs with PituitaryAndTumour
    ica_array[ica_array == 1] = 2
    combined_array = ica_array + pituitary_tumour_array

    # Now explicitly reassign any third class as the second class if necessary
    combined_array[combined_array == 3] = 2

    # Convert array back to image and save
    combined_img = sitk.GetImageFromArray(combined_array)
    combined_img.CopyInformation(ica_img)
    output_path = os.path.join(output_folder, f"{mri_folder}_combined.nii.gz")
    sitk.WriteImage(combined_img, output_path)

    # Checking unique classes for debugging
    unique_classes = np.unique(combined_array)
    print(f"Unique classes in {os.path.basename(mri_folder)} combined image: {unique_classes}")

def process_anatomical_mask_with_itk(mask_path, output_folder):
    # Adjust mask_path to include base_path
    mask_path = os.path.join(base_path, mask_path)
    # Load the mask image
    mask_img = sitk.ReadImage(mask_path)
    mask_array = sitk.GetArrayFromImage(mask_img)

    # Remove class 2 and reassign class 3 to 2
    mask_array[mask_array == 2] = 0
    mask_array[mask_array == 3] = 2

    # Save the modified mask
    modified_mask_img = sitk.GetImageFromArray(mask_array)
    modified_mask_img.CopyInformation(mask_img)
    output_path = os.path.join(output_folder, 'mri0010_modified_anatomical_mask.nii.gz')
    sitk.WriteImage(modified_mask_img, output_path)


# Part 4: Execution - Go through the files in the Google Drive folder
for i in range(1, 11):  # Assuming 10 MRI folders (adjust range if different)
    mri_folder = f"mri000{i}" if i < 10 else f"mri00{i}"
    full_mri_path = os.path.join(base_path, mri_folder)  # Adjusted to correct folder structure

    # Check if it's the special case of mri0010
    if i == 10:
        mask_path = os.path.join(full_mri_path, f'{mri_folder}_anatomical_mask.nii.gz')  # Adjust for the correct file
        process_anatomical_mask_with_itk(mask_path, output_folder)
    else:
        # Process regular MRI folder
        process_mri_with_itk(full_mri_path, output_folder)
print(f"Processed files are saved in {output_folder}")


Unique classes in mri0001 combined image: [0 1 2]
Unique classes in mri0002 combined image: [0 1 2]
Unique classes in mri0003 combined image: [0 1 2]
Unique classes in mri0004 combined image: [0 1 2]
Unique classes in mri0005 combined image: [0 1 2]
Unique classes in mri0006 combined image: [0 1 2]
Unique classes in mri0007 combined image: [0 1 2]
Unique classes in mri0008 combined image: [0 1 2]
Unique classes in mri0009 combined image: [0 1 2]
Processed files are saved in /content/drive/MyDrive/endonasal_mri_new/processed_mri_images
